Notebook qui centralise diverses fonctions communes du projet.

In [0]:
from pyspark.sql import functions as f
from pyspark.sql import DataFrame

def LIN_add_metadata_cols(df: DataFrame) -> DataFrame:
  # Ajouter les colonnes de metadata au dataframe. Les colonnes ajoutées sont: BronzeFileNameSource (le nom du fichier traité), 
  # BronzeFolderSource (chemin du répertoire contenant le fichier traité, démarre et finit avec '/'), IngestionDate (la date UTC de traitement)
  # df (DataFrame), le dataframe auquel il faut ajouter les colonnes.
  # retourne un nouveau dataframe avec les colonnes de metadata. 
  
  currated_df = df.withColumn('absoluteFilePath', f.regexp_replace(f.input_file_name(), r'^abfss://([\w-]+)@([\w-]+).dfs.core.windows.net', ''))
  currated_df = currated_df.withColumn('splittedAbsoluteFilePath', f.split(f.col('absoluteFilePath'), '/'))
  currated_df = currated_df.withColumn('BronzeFileNameSource', f.col('splittedAbsoluteFilePath')[f.size(f.col('splittedAbsoluteFilePath')) - 1])
  currated_df = currated_df.withColumn('BronzeFolderSource', f.expr("concat('/', concat_ws('/', slice(splittedAbsoluteFilePath, 2, size(splittedAbsoluteFilePath)-2)), '/')"))
  currated_df = currated_df.withColumn('IngestionDate', f.current_timestamp())
  return currated_df.drop('absoluteFilePath', 'splittedAbsoluteFilePath')

In [0]:
def LIN_config_adls_gen2_connector(storage_account_name: str, use_service_principal: bool = True ) -> None:
  # Définir la configuration spark pour lire et écrire sur le adls gen2 storage account avec un service principal.
  # storage_account_name (str), storage account auquel s'authentifier.  
  
  if use_service_principal:
    spark.conf.set(f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net", "OAuth")
    spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
    spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account_name}.dfs.core.windows.net", dbutils.secrets.get(scope="scope-datalake",key="sp-adbw-st-datalake-clientid"))
    spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account_name}.dfs.core.windows.net", dbutils.secrets.get(scope="scope-datalake", key="sp-adbw-st-datalake-secret"))
    spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account_name}.dfs.core.windows.net", f"https://login.microsoftonline.com/{dbutils.secrets.get(scope='scope-datalake',key='sp-adbw-st-datalake-tenantid')}/oauth2/token")
  else: 
    spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", "6NNp+F1YV315/lg6oo4si3P83zB149aihPuZha609f6vDRrnHCGgelWflQQT4SUqJlM2kkY0A8O4qVgRlwwzFg==")

In [0]:
from typing import List

def LIN_mount_adls_gen2(containers: List[str], storage_account_name: str) -> None:
  # Monter des containers dans dbfs.
  # containers (List[str]), liste de containers à monter.
  # storage_account_name (str), storage account des containers.
  
  # Configuration de l'authentification.
  configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": dbutils.secrets.get(scope="scope-datalake",key="sp-adbw-st-datalake-clientid"),
           "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="scope-datalake",key="sp-adbw-st-datalake-secret"),
           "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{dbutils.secrets.get(scope='scope-datalake',key='sp-adbw-st-datalake-tenantid')}/oauth2/token"}
  
  # Lister les containers déja montés.
  try:
    mounted_containers = [m.name[:-1] for m in dbutils.fs.ls("/mnt/dlk/")]
  except: 
    mounted_containers = []
    
  # Pour chaque container à monter, vérifier qu'il ne le soit pas déja, sinon le monter.
  for container in containers:
    if container not in mounted_containers:
      dbutils.fs.mount(
        source = f"abfss://{container}@{storage_account_name}.dfs.core.windows.net/",
        mount_point = f"/mnt/dlk/{container}",
        extra_configs = configs)

In [0]:
from typing import List
from pyspark.dbutils import FileInfo

def LIN_get_dbfs_path_size(path: str) -> int:
  # Calculer récursivement la taille totale d'un répertoire du dbfs. Il peut s'agir d'un emplacement ADLS Gen2 si celui-ci est configuré dans les spark.conf.
  # path (str) chemin d'un répertoire dont on veut connaitre la taille.
  # retourne la taille du répertoire.

  def LIN_get_size_recusive(files: List[FileInfo]) -> int:
    # Calculer la taille totale d'une liste de FileInfo.
    # files (List[FileInfo]), liste de FileInfo dont on veut connaitre la taille.
    # retourne la taille de la liste de FileInfo.
    
    total_size = 0
    for file in files:
      if file.isFile():
        total_size += file.size
      else:
        total_size += LIN_get_size_recusive(dbutils.fs.ls(file.path))
    return total_size

  try:
    return LIN_get_size_recusive(dbutils.fs.ls(path))
  except:
    return 0

In [0]:
import re

def LIN_clean_table_name(input_table: str) -> str:
  # Nettoyer le nom d'une table en utilisant une regex.
  # input_table (str), nom de la table à nettoyer.
  # retourne le nom de la table nettoyé.
  
  matches = re.search(r'^(?:\[?(?P<schema>\w+)\]?\.)?\[?(?P<tableName>\w+)\]?$', input_table)
  return matches.group('tableName').upper()

In [0]:
import re

def LIN_parse_datalake_url(url: str) -> (str, str, str):
  # Découper un chemin abfss pour avoir le container, le storage account, le chemin du fichier.
  # url (str), chemin abfss.
  # retourne le container, le storage account, le chemin du fichier.
  
  search_url = re.search(r"^abfss://([\w-]+)@([\w-]+).dfs.core.windows.net(.*)$", url)
  container: str = search_url.group(1)
  storage_account_name: str = search_url.group(2)
  file_path: str = search_url.group(3)
  return container, storage_account_name, file_path

In [0]:
def LIN_run_notebook(name: str, params, context=None) -> int:
  # Démarrer un notebook.
  # name (str), nom du notebook à démarrer.
  # params, paramètres du notebook.
  # context, contexte databricks.
  # retourne 0 si l'exécution n'a pas eu de problème sinon -1.
  
  try:
    if context:
      dbutils.notebook.setContext(context)
    dbutils.notebook.run(name, 0, params)
    return 0
  except:
    return -1